Darknet reposunu clonlayıp set ediyoruz

In [ ]:
#clone darknet repo.
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# GPU ve opencv yi aktif etmek için make dosyasını edit ediyor.
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# CUNDA test komutu.
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make dosyasının çalıştıtrıyor.
!make

Yolov3 clonlanıyor

In [ ]:
# önceden eğitilmiş yolov3 wehits doyasını indiriyor.
!wget https://pjreddie.com/media/files/yolov3.weights

Dosya yüklemek indirmek ve resimleri göstermek için yardımcı fonksiyonlar.

In [ ]:
# yardımcı fonksiyonlar.
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline
 
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
 
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
 
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
 
def download(path):
  from google.colab import files
  files.download(path)

Dataset ile yoloyu eğitmek için ön hazırlıkar.

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# /content/gdrive/My\ Drive/ adresini /mydrive adında bir link oluşturuyor.
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# .zip dosyasını cloud VM e kopyalıyor.
!cp /mydrive/yolov3/obj.zip ../

In [ ]:
# unzip ediyor.
!unzip ../obj.zip -d data/

In [ ]:
# config dosyasını drive yüklüyor ve ismini değiştiriyor.
!cp /darknet/cfg/yolov3.cfg /mydrive/yolov3/yolov3_custom.cfg

In [ ]:
# config dosyasını isteğimize göre ayarladıkdan sonra geri yüklüyoruz.
!cp /mydrive/yolov3/yolov3_custom.cfg ./darknet/cfg

Name isimli dosyanın içeriği daha önce lable için olluşturduğumuz classes dosyası ile aynı olmalı.

Data dosyası ise şu şekilde.

*   classes = 5
*   train = data/train.txt
*   valid = data/test.txt
*   names = data/obj.names
*   backup = /mydrive/yolov3/backup







In [ ]:
# Drive nin içerisinde oluşturduğumuz names ve data isimli doyaları VM e yülüyor.
!cp /mydrive/yolov3/obj.names ./data
!cp /mydrive/yolov3/obj.data  ./data

https://github.com/theAIGuysCode/YoloGenerateTrainingFile/blob/master/generate_train.py/
bu repodaki scripti drive nın içine atıp altaki komutu çalıştırıyoruz.

In [ ]:
# VM e upload ediyoruz scripti.
!cp /mydrive/yolov3/generate_train.py ./

In [ ]:
!python generate_train.py

In [ ]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

googlenin uyumasını engellmek için aşağıdaki javascripti kullanıyoruz.

In [ ]:
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)

In [ ]:
# custom dedectörü eğitmeye başla.
!./darknet/darknet detector train data/obj.data darknet/cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

In [ ]:
!./darknet/darknet detector train data/obj.data darknet/cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights -dont_show

In [ ]:
imShow('chart.png')

RUN Time

In [ ]:
# need to set our custom cfg to test mode 
%cd content/darknet/cfg/
!pwd
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..


In [ ]:
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet/darknet detector test data/obj.data darknet/cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights /mydrive/yolov3/Test.jpg -thresh 0.3
imShow('predictions.jpg')